## 1. Data Cleaning and Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### 1.1. Initial Data Exploration


In [2]:

df = pd.read_csv('../data/loss_details.csv')
print(f"ROWS, COLUMNS : \n {df.shape} \n\n")
print(f"TOP ROWS : \n {df.head()} \n\n")
print(f"COLUMN NAMES : \n {df.columns} \n\n")
print("INFO :")
df.info()
print(f"\n\n DESCRIPTION : \n {df.describe()} \n\n")

ROWS, COLUMNS : 
 (38, 14) 


TOP ROWS : 
   Sl. No.                     State/UT  2018-19  2019-20  % change (YoY)  \
0       1  Andaman and Nicobar Islands     0.11     0.07           -37.0   
1       2               Andhra Pradesh     0.15     1.30           797.0   
2       3            Arunachal Pradesh     0.01     0.05           566.0   
3       4                        Assam     0.81     2.39           195.0   
4       5                        Bihar     0.34     0.63            85.0   

   2020-21  % change (YoY).1  2021-22  % change (YoY).2  2022-23  \
0     0.00             -98.0     0.00            -100.0     0.00   
1     0.72             -45.0     0.29             -59.0     3.30   
2     0.38             720.0     0.00             -99.0     0.00   
3     1.68             -30.0     1.04             -38.0     0.25   
4     1.09              73.0     2.08              90.0     1.20   

   % change (YoY).3  2023-24  % change (YoY).4  2024-25 (Till Sept'24)  
0               Na

### 1.2. Data Cleaning


In [4]:
df = df.drop(columns=['Sl. No.'])
df.rename(columns={
    'State/UT': 'State',
    '2018-19': 'Loss_2018_19',
    '2019-20': 'Loss_2019_20',
    '% change (YoY)': 'YoY_Change_2019_20',
    '2020-21': 'Loss_2020_21',
    '% change (YoY).1': 'YoY_Change_2020_21',
    '2021-22': 'Loss_2021_22',
    '% change (YoY).2': 'YoY_Change_2021_22',
    '2022-23': 'Loss_2022_23',
    '% change (YoY).3': 'YoY_Change_2022_23',
    '2023-24': 'Loss_2023_24',
    '% change (YoY).4': 'YoY_Change_2023_24',
    "2024-25 (Till Sept'24)": 'Loss_2024_25'
}, inplace=True)

In [5]:
df.dropna(thresh=df.shape[1] - 5, inplace=True)

In [6]:
# Recalculate YoY changes
years = ['Loss_2018_19', 'Loss_2019_20', 'Loss_2020_21',
         'Loss_2021_22', 'Loss_2022_23', 'Loss_2023_24', 'Loss_2024_25']

for i in range(1, len(years)):
    prev_col = years[i-1]
    curr_col = years[i]
    target_col = f"YoY_Change_{curr_col.split('_')[1]}_{curr_col.split('_')[2]}"

    df[target_col] = np.where(
        df[prev_col] == 0,
        np.where(df[curr_col] == 0, 0, np.inf),
        ((df[curr_col] - df[prev_col]) / df[prev_col]) * 100
    )

In [7]:
# Fill missing values
loss_cols = [col for col in df.columns if 'Loss_' in col]
df[loss_cols] = df[loss_cols].fillna(0)
yoy_cols = [col for col in df.columns if 'YoY_Change_' in col]
df[yoy_cols] = df[yoy_cols].fillna(0)

print("Data cleaning complete.")
df.info()

Data cleaning complete.
<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 37
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               36 non-null     object 
 1   Loss_2018_19        36 non-null     float64
 2   Loss_2019_20        36 non-null     float64
 3   YoY_Change_2019_20  36 non-null     float64
 4   Loss_2020_21        36 non-null     float64
 5   YoY_Change_2020_21  36 non-null     float64
 6   Loss_2021_22        36 non-null     float64
 7   YoY_Change_2021_22  36 non-null     float64
 8   Loss_2022_23        36 non-null     float64
 9   YoY_Change_2022_23  36 non-null     float64
 10  Loss_2023_24        36 non-null     float64
 11  YoY_Change_2023_24  36 non-null     float64
 12  Loss_2024_25        36 non-null     float64
 13  YoY_Change_2024_25  36 non-null     float64
dtypes: float64(13), object(1)
memory usage: 4.2+ KB


### 1.3. Loading Cleaned Data into MySQL

In [ ]:

db_user = 'root'
db_password = 'Kristine2004$' 
db_host = 'localhost'
db_name = 'banking_analysis'
connection_str = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_str)

df.replace([np.inf, -np.inf], np.nan, inplace=True)

df.to_sql(
    name='banking_losses',
    con=engine,
    if_exists='replace',
    index=False
)
print("loaded to MySQL.")

df.to_csv('banking_losses_for_tableau.csv', index=False)
print("saved to banking_losses_for_tableau.csv")

loaded to MySQL.
saved to banking_losses_cleaned.csv
